In [ ]:
import json
import datetime
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import load_summarize_chain

from langchain_core.documents import Document

# Load environment variables
groq_api_key="#####"


# Initialize LLM and tools
llm = ChatGroq(model="llama3-8b-8192", api_key=groq_api_key)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")

# Departments and contacts
DEPARTMENT_CONTACTS = {
    "Electricity Board": {"phone": "1800-112-233", "email": "power@civic.gov.in"},
    "Department of Water Resources": {"phone": "1800-221-445", "email": "water@civic.gov.in"},
    "Road Development": {"phone": "1800-443-556", "email": "roads@civic.gov.in"},
    "Health Ministry": {"phone": "1800-777-999", "email": "health@civic.gov.in"},
    "Sanitation": {"phone": "1800-333-122", "email": "cleanliness@civic.gov.in"}
}
DEPARTMENTS = list(DEPARTMENT_CONTACTS.keys())

# Classify departments using LLaMA
def classify_departments(text):
    prompt = f"""Given this complaint:
{text}
Classify it into one or more of the following departments:
{', '.join(DEPARTMENTS)}.
Return only department names as a comma-separated list."""
    response = llm.invoke(prompt)
    return [d.strip() for d in response.content.split(",") if d.strip() in DEPARTMENTS]

# Assess severity using LLaMA
def get_severity_score(text):
    prompt = f"""Assess the severity of this civic complaint on a scale of 1 (least) to 5 (most severe).
Complaint:
{text}
Severity (number only):"""
    try:
        response = llm.invoke(prompt).content
        score = int("".join(filter(str.isdigit, response)))
        return min(max(score, 1), 5)
    except Exception:
        return 3

# Summarize using LLaMA
def summarize_text(text):
    docs = [Document(page_content=t) for t in text_splitter.split_text(text)]
    return summarize_chain.run(docs)

# Get contact info
def get_contact_info(departments):
    return {d: DEPARTMENT_CONTACTS[d] for d in departments if d in DEPARTMENT_CONTACTS}

# Fetch suggestions using Serper and LLaMA
def fetch_interim_suggestions(complaint_text, department):
    query = f"answer in few (5-6) short points to, and don't invlove dear sir / madam or any salutaiton - you are a government representaive who received a complain-{complaint_text} answer from perspective of {department} , give answer only nothing else that dont say here is the concise summary or anything , begin like we also show condolences and sorry for incoveniece, try to do this while we resolve it....include pharases like we will surely look into it, we advice you to, don't include things like company or department should do this, instead say we apologise and.."
    #result = search.run(query)
    result=llm.invoke(query).content
    #summary = summarize_text(result)
    return [line.strip("-• ") for line in result.split("\n") if line.strip()]

# Generate officer brief
def generate_officer_brief(summary, severity, departments):
    dept_str = ", ".join(departments) if departments else "relevant authority"
    return f"A complaint has been received regarding {summary}. The issue is rated {severity}/5 in severity and is forwarded to the {dept_str}."

# Process complaint
def process_complaint(text, location):
    timestamp = datetime.datetime.utcnow().isoformat()
    departments = classify_departments(text)
    severity = get_severity_score(text)
    summary = summarize_text(text)
    contact_info = get_contact_info(departments)
    suggestions = []
    for dept in departments:
        suggestions.extend(fetch_interim_suggestions(text, dept))
    officer_brief = generate_officer_brief(summary, severity, departments)

    # Complainer Text Output
    complainer_txt = f"--- COMPLAINER COPY ---\n"
    complainer_txt += f"Original Complaint: {text}\n"
    complainer_txt += f"Location: {location}\n"
    complainer_txt += f"Departments Forwarded: {', '.join(departments)}\n"
    complainer_txt += "Contact Details:\n"
    for dept, info in contact_info.items():
        complainer_txt += f"  {dept}: Phone - {info['phone']}, Email - {info['email']}\n"
    complainer_txt += "Suggestions:\n"
    for s in suggestions:
        complainer_txt += f"  - {s}\n"
    complainer_txt += f"Timestamp: {timestamp}\n"
    complainer_txt += f"Status: Pending\n\n"

    # Officer Text Output
    officer_txt = f"--- OFFICER COPY ---\n"
    officer_txt += f"Timestamp: {timestamp}\n"
    officer_txt += f"Severity: {severity}/5\n"
    officer_txt += f"Summary: {summary}\n"
    officer_txt += f"Location: {location}\n"
    officer_txt += f"Original Complaint: {text}\n"
    officer_txt += f"Departments: {', '.join(departments)}\n\n"

    # Save text outputs
    with open("complainer_output.txt", "a") as f1:
        f1.write(complainer_txt + "\n")

    with open("officer_output.txt", "a") as f2:
        f2.write(officer_txt + "\n")

    # Save JSON outputs
    complainer_json = {
        ##"complaint": text,
        ##"location": location,
        "departments_forwarded": departments,
        "contact_details": contact_info,
        "suggestions": suggestions,
        "timestamp": timestamp,
        #"status": "pending"
    }

    officer_json = {
        "timestamp": timestamp,
        "severity": severity,
        "summary": summary,
        "original_text": text,
        "location": location,
        "departments": departments
    }

    with open("complainer_output.json", "a") as f3:
        json.dump(complainer_json, f3, indent=4)
        f3.write(",\n")

    with open("officer_output.json", "a") as f4:
        json.dump(officer_json, f4, indent=4)
        f4.write(",\n")

    return complainer_txt, officer_txt


if __name__ == "__main__":

    location = input("Enter the location where the issue occurred: ").strip()
    user_input = input("Enter your civic complaint: ").strip()
    if user_input and location:
        complainer_view, officer_view = process_complaint(user_input, location)

        print("\n✅ Complaint Processed!\n")
        print(complainer_view)
        print(officer_view)
    else:
        print("Complaint and location fields cannot be empty.")

